In [ ]:
'''
	Table: patients

	+-------------+---------+
	| Column Name | Type    |
	+-------------+---------+
	| patient_id  | int     |
	| patient_name| varchar |
	| age         | int     |
	+-------------+---------+
	patient_id is the unique identifier for this table.
	Each row contains information about a patient.
	Table: covid_tests

	+-------------+---------+
	| Column Name | Type    |
	+-------------+---------+
	| test_id     | int     |
	| patient_id  | int     |
	| test_date   | date    |
	| result      | varchar |
	+-------------+---------+
	test_id is the unique identifier for this table.
	Each row represents a COVID test result. The result can be Positive, Negative, or Inconclusive.
	Write a solution to find patients who have recovered from COVID - patients who tested positive but later tested negative.

	A patient is considered recovered if they have at least one Positive test followed by at least one Negative test on a later date
	Calculate the recovery time in days as the difference between the first positive test and the first negative test after that positive test
	Only include patients who have both positive and negative test results
	Return the result table ordered by recovery_time in ascending order, then by patient_name in ascending order.

	The result format is in the following example.

 

	Example:

	Input:

	patients table:

	+------------+--------------+-----+
	| patient_id | patient_name | age |
	+------------+--------------+-----+
	| 1          | Alice Smith  | 28  |
	| 2          | Bob Johnson  | 35  |
	| 3          | Carol Davis  | 42  |
	| 4          | David Wilson | 31  |
	| 5          | Emma Brown   | 29  |
	+------------+--------------+-----+
	covid_tests table:

	+---------+------------+------------+--------------+
	| test_id | patient_id | test_date  | result       |
	+---------+------------+------------+--------------+
	| 1       | 1          | 2023-01-15 | Positive     |
	| 2       | 1          | 2023-01-25 | Negative     |
	| 3       | 2          | 2023-02-01 | Positive     |
	| 4       | 2          | 2023-02-05 | Inconclusive |
	| 5       | 2          | 2023-02-12 | Negative     |
	| 6       | 3          | 2023-01-20 | Negative     |
	| 7       | 3          | 2023-02-10 | Positive     |
	| 8       | 3          | 2023-02-20 | Negative     |
	| 9       | 4          | 2023-01-10 | Positive     |
	| 10      | 4          | 2023-01-18 | Positive     |
	| 11      | 5          | 2023-02-15 | Negative     |
	| 12      | 5          | 2023-02-20 | Negative     |
	+---------+------------+------------+--------------+
	Output:

	+------------+--------------+-----+---------------+
	| patient_id | patient_name | age | recovery_time |
	+------------+--------------+-----+---------------+
	| 1          | Alice Smith  | 28  | 10            |
	| 3          | Carol Davis  | 42  | 10            |
	| 2          | Bob Johnson  | 35  | 11            |
	+------------+--------------+-----+---------------+
	Explanation:

	Alice Smith (patient_id = 1):
	First positive test: 2023-01-15
	First negative test after positive: 2023-01-25
	Recovery time: 25 - 15 = 10 days
	Bob Johnson (patient_id = 2):
	First positive test: 2023-02-01
	Inconclusive test on 2023-02-05 (ignored for recovery calculation)
	First negative test after positive: 2023-02-12
	Recovery time: 12 - 1 = 11 days
	Carol Davis (patient_id = 3):
	Had negative test on 2023-01-20 (before positive test)
	First positive test: 2023-02-10
	First negative test after positive: 2023-02-20
	Recovery time: 20 - 10 = 10 days
	Patients not included:
	David Wilson (patient_id = 4): Only has positive tests, no negative test after positive
	Emma Brown (patient_id = 5): Only has negative tests, never tested positive
	Output table is ordered by recovery_time in ascending order, and then by patient_name in ascending order.
'''

In [14]:
import pandas as pd

def find_covid_recovery_patients(patients: pd.DataFrame, covid_tests: pd.DataFrame) -> pd.DataFrame:
    
    covid_tests["test_date"]=pd.to_datetime(covid_tests["test_date"])
    pos_df=(covid_tests[covid_tests["result"]=="Positive"]
            .groupby("patient_id")["test_date"].min().reset_index()
            .rename(columns={"test_date":"pos_date"}))

    neg_df=(covid_tests[covid_tests["result"]=="Negative"])

    df=pos_df.merge(neg_df,on="patient_id").rename(columns={"test_date":"neg_date"})
    df=df[df["neg_date"]>df["pos_date"]]
    df=df.groupby("patient_id").apply(lambda x:x.loc[x["neg_date"].idxmin()]).reset_index(drop=True)

    df["recovery_time"]=(df["neg_date"]-df["pos_date"]).dt.days

    df=(df.merge(patients,on="patient_id")[["patient_id", "patient_name", "age", "recovery_time"]]
        .sort_values(by=["recovery_time","patient_name"]))
    return df

In [15]:
# Patients data
patients_data = {
    'patient_id': [1, 2, 3, 4, 5],
    'patient_name': ['Alice Smith', 'Bob Johnson', 'Carol Davis', 'David Wilson', 'Emma Brown'],
    'age': [28, 35, 42, 31, 29]
}
patients = pd.DataFrame(patients_data)

# COVID tests data
covid_tests_data = {
    'test_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'patient_id': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5],
    'test_date': pd.to_datetime([
        '2023-01-15', '2023-01-25', '2023-02-01', '2023-02-05', '2023-02-12',
        '2023-01-20', '2023-02-10', '2023-02-20', '2023-01-10', '2023-01-18',
        '2023-02-15', '2023-02-20'
    ]),
    'result': [
        'Positive', 'Negative', 'Positive', 'Inconclusive', 'Negative',
        'Negative', 'Positive', 'Negative', 'Positive', 'Positive',
        'Negative', 'Negative'
    ]
}
covid_tests = pd.DataFrame(covid_tests_data)

# invoke function
find_covid_recovery_patients(patients,covid_tests)


C:\Users\MSI\AppData\Local\Temp\ipykernel_2104\1434303565.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby("patient_id").apply(lambda x:x.loc[x["neg_date"].idxmin()]).reset_index(drop=True)


,patient_id,patient_name,age,recovery_time
0,1,Alice Smith,28,10
2,3,Carol Davis,42,10
1,2,Bob Johnson,35,11
